In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import smallgraph as sg
import networkx as nx
from matplotlib import pyplot as plt
import lmz
import eden.graph as eg
import eden.display as ed
from yoda.graphs import ali2graph
import numpy as np
import yoda.ml.simpleMl as sml
import eden.display as ed
from yoda.alignments import load_rfam, filter_by_seqcount

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import normalize
from sklearn.metrics import euclidean_distances
from sklearn.preprocessing import StandardScaler
from kiez import Kiez 
from ubergauss import tools as ut
from ubergauss import optimization as op
import yoda.ml.simpleMl as sml
import yoda.ml.distances as yodadist

In [ ]:
sg.string_to_paramd( yodadist.params)

In [ ]:



from yoda.graphs import alignment_to_vectors

hub_methods =  ['csls', 'dissimlocal', 'localscaling', 'mutualproximity', 'norm', 'no']
hub_methods =  ['csls']

def eval(alis,labels,
         matrix = None,
         ct = .97,
         RYthresh = 0,
         norm = True, 
         d1 = .25,
         d2= .75,
         bad_weight = 0.15,
         fix_edges = True,
         # boost_range = 3, boost_thresh = .5, boost_weight = 1,
         min_r = 2, 
         maxclust =10,
         nest = True,
         simplegraph = False,
         hub_methods=hub_methods,
         clusterSize = 0,
         pca = 0,
         metric = 'euclidean',
         kiezMethod = 'csls',
         kiezK = 5,
         min_d = 1,**nothingtosee):

    if matrix is None:
        matrix = alignment_to_vectors(alis, RYthresh=RYthresh,d1=d1,d2=d2,
                                  fix_edges=fix_edges,ct=ct,bad_weight=bad_weight,
                                  min_r=min_r, min_d= min_d,normalization=norm,
                                  inner_normalization=True,clusterSize=clusterSize,
                                  nest=nest,simplegraph=simplegraph,maxclust=maxclust )

    '''
    # ret= sml.knn_accuracy(matrix,labels)
    # ret = sml.knn_accuracy(matrix,labels)
    dist = 1- (matrix @ matrix.T).toarray()
    dist[dist<0] =0

    def make_score(method,dist,labels):
        if method == 'norm':
            # here we want to just normalize by cloumn... 
            # so we overwrite the distances and tell kiez to not apply itsmethods
            dist = normalize(dist, axis=0) 
            method = 'no' 
        k_inst = Kiez(algorithm='SklearnNN', hubness=method, n_candidates = 40,  algorithm_kwargs= {'metric' : 'precomputed'})
        k_inst.fit(dist)
        dist, neigh_ind = k_inst.kneighbors()
        return sml.average_precision_limited(dist,neigh_ind,labels)
    # print(dist,neigh_ind,labels)
    # ret = sml.average_precision(dist,labels)
    # ret = sml.average_precision_limited(dist,neigh_ind,labels)
    # ret = {method:make_score(method,dist,labels) for method in hub_methods}
    # ret = {'score': ret,   'score_knn': sml.knn_accuracy(matrix,labels,4),  'score_ari': sml.kmeans_ari(matrix, labels)}
    # return ret
    '''
    dist,neigh_ind =  yodadist.mkdistances( matrix, pca, metric, kiezMethod, kiezK)
    return sml.average_precision_limited(dist,neigh_ind,labels)




space = '''min_r 0 3 1
min_d 0 3 1
fix_edges 0 1 1
ct .6 1
d1 0 1
d2 0 1
clusterSize 0 40 5
bad_weight 0 .3'''

# a,l, te = sg.makedata()

#eval(a,l,**{'min_d': 2, 'nest': 1})
a,l = sg.makedata(splits=0)
matrix = alignment_to_vectors(a)


In [ ]:
space = sg.string_to_space(yodadist.params)
parameters = [space.sample() for i in range(50)]
parameters

In [ ]:
results

In [ ]:

#data = sg.makedata(splits=0)
results = op.gridsearch(eval, tasks =  parameters , data=(a,l,matrix), mp=True)

results

In [ ]:
results.sort_values(by='time', ascending=False).head(100)

In [ ]:

#sns.pairplot(results, hue='score')
results.corr()

In [ ]:
%%time
datas  = [sg.makedata(splits=3)[:2]]


# datas   = [sg.makedata(splits=3)[:2] for x in range(3) ]
# datas = [(ali2graph.multiGraph(a),l) for a,l in datas ]


In [ ]:
%%time
parameters = {'clusterSize':10, 'simplegraph':0 , 'maxclust':10, 'fix_edges':True}
eval(*datas[0], **parameters)
# 475

In [ ]:
%%time
parameters = {'clusterSize':[0,15], 'simplegraph':[0,1] , 'maxclust':[10]}
# results = [op.gridsearch(eval, tasks =  parameters , data=data, mp=True)  for data in datas]

results = op.gridsearch(eval, param_dict =  parameters , data=datas[0], mp=True)
# results = [ op.gridsearch(eval, param_dict =  parameters , data=data , mp=True) for data in datas];results = pd.concat(results)

In [ ]:
if False:
    resultsXX = pd.concat(resultsXXX+results4)
    results5c = pd.concat(results5)
    sns.pairplot(resultsXX)
    resultsXX.sort_values(by='csls', ascending=False).head(20)
    sns.lineplot(resultsXX, x= 'clusterSize', y='csls',label= 'conserved columns weighted')
    sns.lineplot(results5c, x= 'clusterSize', y='csls', label='unweighted')

In [ ]:
# results = pd.concat(results)
results.sort_values(by='csls', ascending=False).head(20)

sns.lineplot(results, x= 'clusterSize', y='csls', hue= 'simplegraph')

In [ ]:

sns.pairplot(results, hue='score')


In [ ]:
space_dict = sg.string_to_space(space)
parameters = [space_dict.sample() for i in range(2)]
data = sg.makedata(splits=0)
results = op.gridsearch(eval, tasks =  parameters , data=data, mp=False) 

In [ ]:
results.sort_values(by='csls', ascending=False).head(20)

In [ ]:
plt.figure(figsize=(10, 6))
for column in hub_methods:  # Replace with your actual column names
    sns.kdeplot(data=results[column], label=column)
plt.legend()

In [ ]:


space = '''d1 0 1
d2 0 1
'''

# a,l, te = sg.makedata()

#eval(a,l,**{'min_d': 2, 'nest': 1})
space_dict = sg.string_to_space(space)
parameters = [space_dict.sample() for i in range(100)]
data = sg.makedata(splits=0)
for t in parameters:
    t['hub_methods']=['csls']
results = op.gridsearch(eval, tasks =  parameters , data=data, mp=True) 
results.sort_values(by='csls', ascending=False).head(100)

In [ ]:
sns.pairplot(results)

In [ ]:
results.sort_values(by='csls', ascending=False).head(100)

In [ ]:

def rank_columns_and_average(df, columns):
    # Subset the dataframe to include only the specified columns
    subset_df = df[columns]
    # Rank the values row-wise
    ranked_df = subset_df.rank(axis=1)
    # Calculate the average rank for each column
    avg_ranks = ranked_df.mean().to_dict()
    # Create a list of pairs (column name, average rank)
    result = [(col, avg_ranks[col]) for col in columns]
    return result


rank_columns_and_average(results,['csls', 'dissimlocal', 'localscaling', 'mutualproximity', 'norm', 'no'])

In [ ]:
data_all = [sg.makedata() for i in range(REPEATS)]
datasets = [d[:2] for d in data_all]
test_sets = [d[2] for d in data_all]


In [ ]:
test_sets, datasets = datasets, test_sets

In [ ]:
space = sg.string_to_space(space)
parameters = [space.sample() for i in range(PARAMS)]

In [ ]:

results = [ op.gridsearch(eval, parameters , tasks = parameters , data= [a,l], mp=True)  for a,l in datasets]


In [ ]:
def train_test_error(results):
    for i,df in enumerate(results):
        # in all the dfs, we only look at the max value up to index
        df['max_score'] = df['score'].expanding().max()
        # when a value changes, calculate the test error 
        score_changes = df['max_score'].diff().ne(0)
        top_score_rows = df[score_changes]
        top_score_dicts = top_score_rows.to_dict('records')
        nures =  op.gridsearch(eval, parameters , tasks = top_score_dicts , data= test_sets[i], mp=True) 


        def outer_pointget():
            idx = -1
            def pointget(z):
                nonlocal idx
                if z:
                    idx += 1
                return nures['score'][idx]
            return pointget
        pointget= outer_pointget()     
        df['test_score'] = [ pointget(e) for e in score_changes ] 

train_test_error(results)


In [ ]:
# make a lineplot 
sns.set_theme()
sns.set_context("talk")

merged_df = pd.concat(results)
sns.lineplot(data = merged_df,x=merged_df.index, y = 'max_score', label ='train score')
sns.lineplot(data = merged_df,x=merged_df.index, y = 'test_score', label ='test score')

In [ ]:
df  = merged_df
top_score_idx = df['score'].idxmax()
top_score_row = df.loc[top_score_idx]
top_score_dict = top_score_row.to_dict()

In [ ]:
top_score_dict

In [ ]:
import pandas as pd
x = 5

top_dfs = [df.sort_values('score', ascending=False).head(x) for df in results]
merged_df = pd.concat(top_dfs)
merged_df

In [ ]:
merged_df = pd.concat(results)
merged_df.corr()

In [ ]:
df.groupby('min_r')['score'].mean()